In [63]:
import numpy as np
import pandas as pd
from collections import Counter
import math

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.grid_search import GridSearchCV
from sklearn import svm
from sklearn.decomposition import RandomizedPCA
from sklearn.metrics import classification_report
from sklearn import preprocessing
import matplotlib.pyplot as plt
import scipy.optimize as opt
import glob
import os
import sys
import pickle
import time
import timeit

from sklearn.decomposition import RandomizedPCA

import theano
import theano.tensor as T
import lasagne

#np.random.seed(1337)  # for reproducibility
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, Layer, Reshape, AutoEncoder, Merge
from keras.layers.convolutional import Convolution2D,MaxPooling2D,Convolution1D,MaxPooling1D,UpSampling2D, ZeroPadding2D
from keras.optimizers import SGD, Adam, RMSprop, Adadelta
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
import keras.utils.visualize_util as vutil
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from keras.regularizers import l1, l2, l1l2, activity_l2, activity_l1
import keras.layers.containers as containers
from keras.layers.noise import GaussianNoise
from keras.regularizers import ActivityRegularizer
from keras import backend as K
import keras.utils.visualize_util as vutil
from keras.models import model_from_json

import seaborn as sns

%matplotlib inline

from IPython.display import SVG

import h5py

np.set_printoptions(suppress=True)  #Supress exponent of the number 2e-2 == 0.02

%matplotlib inline

In [64]:
#PCSI0046

In [65]:
#Read the data

GenomicFeatures_PCSI0046 = pd.read_csv(open('data/PCSI0046/GenomicFeatures_PCSI0046.txt','r'),sep='\t')
SNVCalls_PCSI0046    = pd.read_csv(open('data/PCSI0046/SNVCalls_PCSI0046.txt','r'),sep='\t') 

In [66]:
print GenomicFeatures_PCSI0046.shape
print SNVCalls_PCSI0046.shape

(103207, 15)
(103207, 129)


In [67]:
np.sum(GenomicFeatures_PCSI0046.isnull())

CHROM                  0
POS                    0
RefAllele              0
NonRefAllele           0
BaseQual               0
TumourCoverage         0
NormalCoverage         0
MapQual                0
ReadPosition         145
Trinucleotide          0
HomopolymerRate        0
GCcontent              0
GenomicLocation    24849
DistanceSNP            0
StrandBias           145
dtype: int64

In [68]:
GenomicFeatures_PCSI0046['ReadPosition'].fillna(0, inplace=True)  #Replace na with 0
GenomicFeatures_PCSI0046['StrandBias'].fillna(0, inplace=True)

#print np.sum(GenomicFeatures_PCSI0023.isnull())

In [69]:
np.unique(GenomicFeatures_PCSI0046['GenomicLocation'])

array(['UTR3', 'UTR5', 'downstream', 'exonic', 'exonic;splicing',
       'intergenic', 'intronic', 'nan', 'ncRNA_UTR3', 'ncRNA_UTR5',
       'ncRNA_exonic', 'ncRNA_intronic', 'splicing', 'upstream',
       'upstream;downstream'], 
      dtype='|S19')

In [70]:
GenomicFeatures_PCSI0046.head()

,CHROM,POS,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias
0,chr1,14933,112,44,4747,156,128,23,55.0,ACC,3.139535,0.592040,ncRNA_intronic,15990,0.416667
1,chr1,14976,107,39,4243,148,110,23,49.5,GCG,2.817518,0.582090,ncRNA_exonic,15947,0.533784
2,chr1,28588,15,1,446,17,16,20,87.0,ACC,4.192593,0.313433,ncRNA_intronic,2335,0.588235
3,chr1,49298,63,7,2258,70,71,23,39.0,ATA,2.256757,0.452736,intergenic,12144,0.485714
4,chr1,77462,6,20,799,26,29,15,47.0,ACG,3.261538,0.258706,intergenic,5622,0.730769


In [71]:
SNVCalls_PCSI0046.head()

,CHROM,POS,END,X2399009,X2642559,X2642561,X2673219,X2673233,X2701053,X2701056,X2811094,X2811096,X2811100,X2811102,X2811395,X2812912,X2813251,X2813261,X2813343,X2813383,
0,chr1,14932,14933,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...
1,chr1,14975,14976,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...
2,chr1,28587,28588,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
3,chr1,49297,49298,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...
4,chr1,77461,77462,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...


In [72]:
np.unique(GenomicFeatures_PCSI0046['Trinucleotide'])

array(['ACA', 'ACC', 'ACG', 'ACT', 'ATA', 'ATC', 'ATG', 'ATT', 'CCA',
       'CCC', 'CCG', 'CCT', 'CTA', 'CTC', 'CTG', 'CTT', 'GCA', 'GCC',
       'GCG', 'GCT', 'GTA', 'GTC', 'GTG', 'GTT', 'TCA', 'TCC', 'TCG',
       'TCT', 'TTA', 'TTC', 'TTG', 'TTT'], 
      dtype='|S3')

In [73]:
PCSI0046 = pd.merge(GenomicFeatures_PCSI0046, SNVCalls_PCSI0046,left_on=['CHROM', 'POS'], right_on=['CHROM','END'])


In [74]:
PCSI0046.head() #182+15

,CHROM,POS_x,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias,POS_y,END,X2399009,X2642559,X2642561,
0,chr1,14933,112,44,4747,156,128,23,55.0,ACC,3.139535,0.592040,ncRNA_intronic,15990,0.416667,14932,14933,0,0,0,...
1,chr1,14976,107,39,4243,148,110,23,49.5,GCG,2.817518,0.582090,ncRNA_exonic,15947,0.533784,14975,14976,0,0,0,...
2,chr1,28588,15,1,446,17,16,20,87.0,ACC,4.192593,0.313433,ncRNA_intronic,2335,0.588235,28587,28588,0,0,0,...
3,chr1,49298,63,7,2258,70,71,23,39.0,ATA,2.256757,0.452736,intergenic,12144,0.485714,49297,49298,0,0,0,...
4,chr1,77462,6,20,799,26,29,15,47.0,ACG,3.261538,0.258706,intergenic,5622,0.730769,77461,77462,0,0,0,...


In [75]:
PCSI0046_X = PCSI0046.iloc[:,2:15] #Exclude CHROM, POS
PCSI0046_Y = PCSI0046.iloc[:,17:]  #Exclude CHROM_y, POS_y, END

In [76]:
print PCSI0046_X.shape
print PCSI0046_Y.shape

(103207, 13)
(103207, 126)


In [77]:
PCSI0046_X[:2]

,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias
0,112,44,4747,156,128,23,55.0,ACC,3.139535,0.59204,ncRNA_intronic,15990,0.416667
1,107,39,4243,148,110,23,49.5,GCG,2.817518,0.58209,ncRNA_exonic,15947,0.533784


In [78]:
PCSI0046_Y[:2]

,X2399009,X2642559,X2642561,X2673219,X2673233,X2701053,X2701056,X2811094,X2811096,X2811100,X2811102,X2811395,X2812912,X2813251,X2813261,X2813343,X2813383,X2813451,X2813727,X2813730,
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...


In [79]:
totalSample = PCSI0046_Y.shape[0]
PCSI0046_Y_Val = np.zeros((totalSample,1)) #Will be our label now

print totalSample
print PCSI0046_Y_Val.shape

103207
(103207, 1)


In [80]:
PCSI0046_Y.shape

(103207, 126)

In [81]:
PCSI0046_Y.values[0].shape

(126,)

In [82]:
for i in range(totalSample):
    PCSI0046_Y_Val[i] = np.argmax(np.bincount(PCSI0046_Y.values[i]))

In [83]:
PCSI0046_Y_Val_1 = pd.DataFrame(PCSI0046_Y_Val)
PCSI0046_Y_Val_1.columns = ['Label']

In [84]:
PCSI0046_Y_Val_1[:3]

,Label
0,0
1,0
2,0


In [85]:
PCSI0046_dataframe = pd.concat([PCSI0046_X, PCSI0046_Y_Val_1],axis=1)

In [86]:
PCSI0046_dataframe.shape

(103207, 14)

In [87]:
PCSI0046_dataframe[:2]

,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias,Label
0,112,44,4747,156,128,23,55.0,ACC,3.139535,0.59204,ncRNA_intronic,15990,0.416667,0
1,107,39,4243,148,110,23,49.5,GCG,2.817518,0.58209,ncRNA_exonic,15947,0.533784,0


In [88]:
PCSI0046_dataframe.to_csv('PCSI0046.csv')

In [27]:
#Now we have variable importance ...lets do analysis

In [28]:
%load_ext rpy2.ipython

In [29]:
%%R
# Read in the data
PCSI0046Frame = read.csv("PCSI0046.csv")
str(PCSI0046Frame)

'data.frame':	103207 obs. of  15 variables:
 $ X              : int  0 1 2 3 4 5 6 7 8 9 ...
 $ RefAllele      : int  112 107 15 63 6 2 47 83 78 57 ...
 $ NonRefAllele   : int  44 39 1 7 20 12 12 8 9 10 ...
 $ BaseQual       : int  4747 4243 446 2258 799 449 1847 2919 2781 1528 ...
 $ TumourCoverage : int  156 148 17 70 26 14 62 92 88 67 ...
 $ NormalCoverage : int  128 110 16 71 29 29 49 127 115 69 ...
 $ MapQual        : num  23 23 20 23 15 19 0 29 0 37 ...
 $ ReadPosition   : num  55 49.5 87 39 47 60 53 36 48 30 ...
 $ Trinucleotide  : Factor w/ 32 levels "ACA","ACC","ACG",..: 2 19 2 5 3 29 1 13 25 12 ...
 $ HomopolymerRate: num  3.14 2.82 4.19 2.26 3.26 ...
 $ GCcontent      : num  0.592 0.582 0.313 0.453 0.259 ...
 $ GenomicLocation: Factor w/ 15 levels "","downstream",..: 8 7 8 5 5 5 5 5 5 5 ...
 $ DistanceSNP    : int  15990 15947 2335 12144 5622 673 5796 6707 6730 13330 ...
 $ StrandBias     : num  0.417 0.534 0.588 0.486 0.731 ...
 $ Label          : num  0 0 0 0 0 0 0 0 0 0 .

In [30]:
%%R
#sum(is.na(CPCG0183Frame$StrandBias))
#sum(is.na(IS1Frame$ReadPosition)) 
sum(is.na(PCSI0046Frame)) 

[1] 0


In [31]:
#%%R
#CPCG0235Frame$ReadPosition[is.na(CPCG0235Frame$ReadPosition)] = mean(CPCG0235Frame$ReadPosition, na.rm=TRUE)
#CPCG0235Frame$StrandBias[is.na(CPCG0235Frame$StrandBias)] = mean(CPCG0235Frame$StrandBias, na.rm=TRUE)

#print (sum(is.na(CPCG0235Frame$ReadPosition)) )
#print (sum(is.na(CPCG0235Frame$StrandBias)) )

In [32]:
%%R
sum(is.na(PCSI0046Frame))  #No NA value now

[1] 0


In [33]:
%%R
write.csv(PCSI0046Frame, file = "PCSI0046_imputed.csv")  #Cleaned CSV without NA

In [34]:
%%R
library(h2o)
localH2O = h2o.init(ip = "localhost", port = 54321, startH2O = TRUE,min_mem_size = "3g")

/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: statmod

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: 
----------------------------------------------------------------------

Your next step is to start H2O:
    > h2o.init()

For H2O package documentation, ask for help:
    > ??h2o

After starting H2O, you can use the Web UI at http://localhost:54321
For more information visit http://docs.h2o.ai

----------------------------------------------------------------------


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: 
Attaching package: ‘h2o’


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: The following objects are masked from ‘

 Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         1 hours 12 minutes 
    H2O cluster version:        3.9.1.3435 
    H2O cluster name:           H2O_started_from_R_rbhat_xqb196 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   26.57 GB 
    H2O cluster total cores:    56 
    H2O cluster allowed cores:  2 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    R Version:                  R version 3.3.0 beta (2016-03-30 r70404) 



In [35]:
%%R
## Import CSV as H2O:
csvPath = "/home/rbhat/Workspace/S3Lab_Projects/SMC-MetaChallange/PCSI0046_imputed.csv"
PCSI0046.hex          = h2o.importFile(path = csvPath, destination_frame = "PCSI0046.hex") 
train                 = as.data.frame(PCSI0046.hex)

str(train)

  |======================================================================| 100%
'data.frame':	103207 obs. of  16 variables:
 $ C1             : int  1 2 3 4 5 6 7 8 9 10 ...
 $ X              : int  0 1 2 3 4 5 6 7 8 9 ...
 $ RefAllele      : int  112 107 15 63 6 2 47 83 78 57 ...
 $ NonRefAllele   : int  44 39 1 7 20 12 12 8 9 10 ...
 $ BaseQual       : int  4747 4243 446 2258 799 449 1847 2919 2781 1528 ...
 $ TumourCoverage : int  156 148 17 70 26 14 62 92 88 67 ...
 $ NormalCoverage : int  128 110 16 71 29 29 49 127 115 69 ...
 $ MapQual        : num  23 23 20 23 15 19 0 29 0 37 ...
 $ ReadPosition   : num  55 49.5 87 39 47 60 53 36 48 30 ...
 $ Trinucleotide  : Factor w/ 32 levels "ACA","ACC","ACG",..: 2 19 2 5 3 29 1 13 25 12 ...
 $ HomopolymerRate: num  3.14 2.82 4.19 2.26 3.26 ...
 $ GCcontent      : num  0.592 0.582 0.313 0.453 0.259 ...
 $ GenomicLocation: Factor w/ 14 levels "downstream","exonic",..: 7 6 7 4 4 4 4 4 4 4 ...
 $ DistanceSNP    : int  15990 15947 2335 12144 562

In [36]:
%%R

train$Trinucleotide   = as.factor(train$Trinucleotide)
train$GenomicLocation = as.factor(train$GenomicLocation)
train$Label           = as.factor(train$Label)

train_h2o             = as.h2o(train)

  |======================================================================| 100%


In [37]:
%%R
#Training a deep learning model:---------------------------------------------------------#
model <- h2o.deeplearning( x = 3:15, # column numbers for predictors
                           y = 16,            # column number for label
                           training_frame = train_h2o,
                           activation = "RectifierWithDropout", 
                           input_dropout_ratio = 0.2,
                           hidden_dropout_ratios = c(0.5,0.5),
                           balance_classes = TRUE, 
                           hidden = c(10,8),  ## two hidden layers
                           epochs = 50,
                           variable_importances = T)

  |======================================================================| 100%


In [38]:
%%R
var_importance = h2o.varimp(model)
#print(var_importance)
h2o.varimp(model)

Variable Importances: 
        variable relative_importance scaled_importance percentage
1   NonRefAllele            1.000000          1.000000   0.091963
2      RefAllele            0.595051          0.595051   0.054723
3 NormalCoverage            0.516878          0.516878   0.047534
4 TumourCoverage            0.324426          0.324426   0.029835
5        MapQual            0.319343          0.319343   0.029368

---
                     variable relative_importance scaled_importance percentage
54   GenomicLocation.intronic            0.071297          0.071297   0.006557
55               ReadPosition            0.069421          0.069421   0.006384
56 GenomicLocation.intergenic            0.059001          0.059001   0.005426
57                 StrandBias            0.057090          0.057090   0.005250
58                  GCcontent            0.055294          0.055294   0.005085
59  Trinucleotide.missing(NA)            0.000000          0.000000   0.000000


In [39]:
%%R
write.table(var_importance, file="PCSI0046_variable_importance.csv")

In [40]:
var_importance = pd.read_csv(open('PCSI0046_variable_importance.csv','r'),sep=' ')
#pd.read_csv('CPCG0100_variable_importance.csv')

In [41]:
var_importance[:5]

,variable,relative_importance,scaled_importance,percentage
1,NonRefAllele,1.000000,1.000000,0.091963
2,RefAllele,0.595051,0.595051,0.054723
3,NormalCoverage,0.516878,0.516878,0.047534
4,TumourCoverage,0.324426,0.324426,0.029835
5,MapQual,0.319343,0.319343,0.029368


In [42]:
GenomicFeatures_PCSI0046[:5]

,CHROM,POS,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias
0,chr1,14933,112,44,4747,156,128,23,55.0,ACC,3.139535,0.592040,ncRNA_intronic,15990,0.416667
1,chr1,14976,107,39,4243,148,110,23,49.5,GCG,2.817518,0.582090,ncRNA_exonic,15947,0.533784
2,chr1,28588,15,1,446,17,16,20,87.0,ACC,4.192593,0.313433,ncRNA_intronic,2335,0.588235
3,chr1,49298,63,7,2258,70,71,23,39.0,ATA,2.256757,0.452736,intergenic,12144,0.485714
4,chr1,77462,6,20,799,26,29,15,47.0,ACG,3.261538,0.258706,intergenic,5622,0.730769


In [43]:
var_imp_dict = {}

for i in range(var_importance.shape[0]):
    var_imp_dict[var_importance.iloc[i,0]] = var_importance.iloc[i,1]

var_imp_dict

{'BaseQual': 0.27338093519210799,
 'DistanceSNP': 0.27333459258079501,
 'GCcontent': 0.055294193327426903,
 'GenomicLocation.UTR3': 0.21927419304847698,
 'GenomicLocation.UTR5': 0.16523063182830802,
 'GenomicLocation.downstream': 0.19278885424137102,
 'GenomicLocation.exonic': 0.18935355544090302,
 'GenomicLocation.exonic;splicing': 0.16265919804573098,
 'GenomicLocation.intergenic': 0.059000521898269702,
 'GenomicLocation.intronic': 0.071297012269496904,
 'GenomicLocation.missing(NA)': 0.14783723652362804,
 'GenomicLocation.ncRNA_UTR3': 0.14071482419967699,
 'GenomicLocation.ncRNA_UTR5': 0.19737727940082603,
 'GenomicLocation.ncRNA_exonic': 0.19220119714736897,
 'GenomicLocation.ncRNA_intronic': 0.19683381915092499,
 'GenomicLocation.splicing': 0.165991947054863,
 'GenomicLocation.upstream': 0.245448052883148,
 'GenomicLocation.upstream;downstream': 0.151036307215691,
 'HomopolymerRate': 0.20242008566856398,
 'MapQual': 0.31934335827827504,
 'NonRefAllele': 1.0,
 'NormalCoverage': 0.5

In [44]:
GenomicFeatures_PCSI0046.columns

Index([u'CHROM', u'POS', u'RefAllele', u'NonRefAllele', u'BaseQual', u'TumourCoverage', u'NormalCoverage', u'MapQual', u'ReadPosition', u'Trinucleotide', u'HomopolymerRate', u'GCcontent', u'GenomicLocation', u'DistanceSNP', u'StrandBias'], dtype='object')

In [45]:
GenomicFeatures_PCSI0046.dtypes

CHROM               object
POS                  int64
RefAllele            int64
NonRefAllele         int64
BaseQual             int64
TumourCoverage       int64
NormalCoverage       int64
MapQual            float64
ReadPosition       float64
Trinucleotide       object
HomopolymerRate    float64
GCcontent          float64
GenomicLocation     object
DistanceSNP          int64
StrandBias         float64
dtype: object

In [46]:
GenomicFeatures_PCSI0046.iloc[0,13]

15990

In [47]:
GenomicFeatures_PCSI0046[:2]
#print GenomicFeatures_IS1.iloc[0,2]

,CHROM,POS,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias
0,chr1,14933,112,44,4747,156,128,23,55.0,ACC,3.139535,0.59204,ncRNA_intronic,15990,0.416667
1,chr1,14976,107,39,4243,148,110,23,49.5,GCG,2.817518,0.58209,ncRNA_exonic,15947,0.533784


In [48]:
noOfSample = GenomicFeatures_PCSI0046.shape[0]

PCSI0046_Dict = {}

for i in range(noOfSample):
    
    pos = GenomicFeatures_PCSI0046.iloc[i,1] #gets the pos value
    
    RefAllele      = GenomicFeatures_PCSI0046.iloc[i,2]*var_imp_dict['RefAllele']
    NonRefAllele   = GenomicFeatures_PCSI0046.iloc[i,3]*var_imp_dict['NonRefAllele']
    BaseQual       = GenomicFeatures_PCSI0046.iloc[i,4]*var_imp_dict['BaseQual']
    TumourCoverage = GenomicFeatures_PCSI0046.iloc[i,5]*var_imp_dict['TumourCoverage']
    NormalCoverage = GenomicFeatures_PCSI0046.iloc[i,6]*var_imp_dict['NormalCoverage']
    MapQual        = GenomicFeatures_PCSI0046.iloc[i,7]*var_imp_dict['MapQual']
    ReadPosition   = GenomicFeatures_PCSI0046.iloc[i,8]*var_imp_dict['ReadPosition']
    
    Trinucleotide  = 0
    if isinstance(GenomicFeatures_PCSI0046.iloc[i,9], str):
        Trinucleotide  = var_imp_dict['Trinucleotide.'+str(GenomicFeatures_PCSI0046.iloc[i,9])]
    HomopolymerRate= GenomicFeatures_PCSI0046.iloc[i,10]*var_imp_dict['HomopolymerRate']
    GCcontent      = GenomicFeatures_PCSI0046.iloc[i,11]*var_imp_dict['GCcontent']
    
    GenomicLocation = 0
    if isinstance(GenomicFeatures_PCSI0046.iloc[i,12], str):
        GenomicLocation= var_imp_dict['GenomicLocation.'+str(GenomicFeatures_PCSI0046.iloc[i,12])]
    DistanceSNP    = GenomicFeatures_PCSI0046.iloc[i,13]*var_imp_dict['DistanceSNP']
    StrandBias     = GenomicFeatures_PCSI0046.iloc[i,14]*var_imp_dict['StrandBias']
    
    #print 'RefAllele=',RefAllele,' NonRefAllele=',NonRefAllele,' BaseQual=',BaseQual,' TumourCoverage=',TumourCoverage
    #print 'NormalCoverage=',NormalCoverage,' MapQual=',MapQual,' ReadPosition=',ReadPosition,' Trinucleotide=',Trinucleotide
    #print 'HomopolymerRate=',HomopolymerRate,' GCcontent=',GCcontent,' GenomicLocation=',GenomicLocation,' Distance=',Distance
    #print 'StrandBias=',StrandBias
    PCSI0046_Dict[pos] = RefAllele+NonRefAllele+BaseQual+TumourCoverage+NormalCoverage+MapQual+ReadPosition+Trinucleotide+ \
                         HomopolymerRate+GCcontent+GenomicLocation+DistanceSNP+StrandBias

In [56]:
#Sorting dictionary
PCSI0046_Dict1 = [(k,v) for v,k in sorted([(v,k) for k,v in PCSI0046_Dict.items()],reverse=True)]

In [57]:
PCSI0046_Dict_Frame = pd.DataFrame(PCSI0046_Dict1, columns=['POS', 'Value'])

In [58]:
PCSI0046_Dict_Frame.to_csv("PCSI0046_Dict_Frame.csv")

In [89]:
PCSI0046_Dict_Frame = pd.read_csv("PCSI0046_Dict_Frame.csv")

In [90]:
PCSI0046_Dict_Frame[:10]

,Unnamed: 0,POS,Value
0,0,155565,696097.877652
1,1,157281,695855.223338
2,2,155944,695459.257273
3,3,166699,692610.243302
4,4,215423,679751.475047
5,5,369223,637127.984703
6,6,369247,637094.253251
7,7,417350,623975.481281
8,8,477937,607470.130888
9,9,545609,588904.932254


In [91]:
GenomicFeatures_PCSI0046_1 = pd.concat([PCSI0046.iloc[:,0:16], PCSI0046_Y_Val_1],axis=1)

In [92]:
rows = PCSI0046_Dict_Frame['POS'][:20].values
print rows
GenomicFeatures_PCSI0046_Top20 = GenomicFeatures_PCSI0046_1.loc[GenomicFeatures_PCSI0046_1['POS_x'].isin(rows)]

[155565 157281 155944 166699 215423 369223 369247 417350 477937 545609
 579444 646304 702347 740485 740489 745914 745916 806891 810875 878688]


In [94]:
GenomicFeatures_PCSI0046_Top20[:10]

,CHROM,POS_x,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias,POS_y,Label
94937,chrX,155565,54,4,1843,97,92,0,71,TCT,1.919255,0.487562,intergenic,2544403,0.556701,155564,0
94938,chrX,155944,8,0,219,9,30,0,84,ACT,1.924051,0.497512,intergenic,2544024,0.555556,155943,0
94939,chrX,157281,83,4,2577,114,101,0,47,GCT,1.855346,0.487562,NaN,2542687,0.543860,157280,0
94940,chrX,166699,14,2,523,21,26,0,83,CCT,3.066667,0.686567,intergenic,2533269,0.619048,166698,0
94941,chrX,215423,63,2,1997,67,58,0,21,TTA,2.493151,0.462687,intronic,2484545,0.388060,215422,0
94942,chrX,369223,3,2,119,8,18,0,84,CCG,1.955975,0.582090,intergenic,2330745,0.750000,369222,0
94943,chrX,369247,2,0,56,4,11,0,48,ACG,2.000000,0.577114,intergenic,2330721,0.750000,369246,0
94944,chrX,417350,2,5,107,20,24,0,30,ATC,1.824242,0.373134,intergenic,2282618,0.650000,417349,0
94945,chrX,477937,10,0,317,11,18,0,80,GCA,4.661538,0.412935,intergenic,2222031,0.636364,477936,0
94946,chrX,545609,3,0,99,6,14,0,74,CTG,3.181102,0.552239,intergenic,2154359,0.500000,545608,0


In [62]:
GenomicFeatures_PCSI0046_Top20.to_csv('GenomicFeatures_PCSI0046_Top20.csv')